# Recife — Análise Avançada de Risco (Alagamento/Deslizamento)
Este notebook amplia o notebook inicial com: carregamento flexível de dados, engenharia de features temporais (lags, janelas rolantes), validação cruzada com RandomForest/XGBoost, busca de hiperparâmetros leve, calibração, explicabilidade (SHAP quando disponível) e visualização espacial (folium com fallback).

In [ ]:
# Imports principais
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix)
import joblib
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10,6)

In [ ]:
# Tentar importar libs opcionais (XGBoost, shap, folium). Se não existirem, usar fallback e avisar.
optional = {}
try:
    import xgboost as xgb
    optional['xgboost'] = True
except Exception as e:
    optional['xgboost'] = False
    print('xgboost não disponível; pulará modelos XGBoost')

try:
    import shap
    optional['shap'] = True
except Exception as e:
    optional['shap'] = False
    print('shap não disponível; pulará explicabilidade SHAP')

try:
    import folium
    optional['folium'] = True
except Exception as e:
    optional['folium'] = False
    print('folium não disponível; pulará mapa interativo')

In [ ]:
# Carregamento flexível de dados: tenta várias localizações comuns do projeto e, se falhar, gera dados sintéticos compatíveis
base = Path('..') / 'data' / 'processed'
candidates = [
    base / 'real_data_converted.csv',
    base / '2024.csv',
    base / 'simulated_daily.csv',
    base / 'Dados pluviométricos da APAC - Região metropolitana de Recife - 2024.csv'
]
df = None
for p in candidates:
    if p.exists():
        print('Carregando', p)
        try:
            df = pd.read_csv(p, parse_dates=['date'], dayfirst=True)
            break
        except Exception as e:
            print('Falha ao ler', p, e)

if df is None:
    # Gerar base sintética (fallback)
    print('Nenhum arquivo encontrado — gerando dados sintéticos de fallback')
    neighborhoods = ['Boa Viagem','Espinheiro','Casa Amarela','Ibura','Várzea','Santo Amaro','Centro','Aflitos','Pina','Cordeiro']
    date_rng = pd.date_range(start='2024-04-01', end='2024-04-14', freq='6H')
    rng = np.random.default_rng(42)
    rows = []
    for nb in neighborhoods:
        for dt in date_rng:
            base_r = 10 if nb in ['Ibura','Várzea','Pina'] else 3
            rain = float(max(0, base_r + rng.normal(scale=8)))
            base_t = 120 if nb in ['Boa Viagem','Pina','Aflitos'] else 60
            tide = float(max(0, base_t + rng.normal(scale=40)))
            rows.append({'date': dt, 'neighborhood': nb, 'rain_mm': rain, 'tide_cm': tide})
    df = pd.DataFrame(rows)
    vuln_scores = {'Boa Viagem':0.3,'Espinheiro':0.4,'Casa Amarela':0.6,'Ibura':0.8,'Várzea':0.9,'Santo Amaro':0.5,'Centro':0.7,'Aflitos':0.35,'Pina':0.65,'Cordeiro':0.55}
    vuln_df = pd.DataFrame([{'neighborhood':k,'vuln_index':v} for k,v in vuln_scores.items()])
    df = df.merge(vuln_df, on='neighborhood', how='left')

# Visualizar primeiras linhas
print('df shape:', df.shape)
df.head()

In [ ]:
# Normalização básica e parsing de datas — garantir colunas necessárias
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
else:
    raise ValueError('Coluna date ausente')

for col in ['rain_mm','tide_cm']:
    if col not in df.columns:
        df[col] = 0.0

if 'vuln_index' not in df.columns:
    df['vuln_index'] = df.groupby('neighborhood')['rain_mm'].transform('median') * 0 + 0.5

# ordenar e indexar por bairro para operações rolling por grupo
df = df.sort_values(['neighborhood','date']).reset_index(drop=True)

# Engenharia: lags e janelas rolantes (6H freq assumida => 4 períodos = 24h)
df['rain_24h_sum'] = df.groupby('neighborhood')['rain_mm'].rolling(window=4, min_periods=1).sum().reset_index(level=0, drop=True)
df['rain_48h_sum'] = df.groupby('neighborhood')['rain_mm'].rolling(window=8, min_periods=1).sum().reset_index(level=0, drop=True)
df['tide_12h_max'] = df.groupby('neighborhood')['tide_cm'].rolling(window=2, min_periods=1).max().reset_index(level=0, drop=True)
# lags
df['rain_lag_1'] = df.groupby('neighborhood')['rain_mm'].shift(1).fillna(0)

# features temporais
df['hour'] = df['date'].dt.hour
df['dayofweek'] = df['date'].dt.dayofweek

# risk index heurístico (similar ao original) — útil como target proxy quando não existir label real
df['rain_norm'] = df['rain_24h_sum'] / (df['rain_24h_sum'].max() + 1e-9)
df['tide_norm'] = df['tide_12h_max'] / (df['tide_12h_max'].max() + 1e-9)
df['vuln_norm'] = df['vuln_index'] / df['vuln_index'].max()
df['risk_index'] = 0.5 * df['rain_norm'] + 0.35 * df['tide_norm'] + 0.15 * df['vuln_norm']
# Classe binária para risco (proxy). Ajuste threshold conforme necessidade.
df['risk_label'] = (df['risk_index'] > df['risk_index'].quantile(0.75)).astype(int)

print('Features criadas. Exemplo:')
df.head()

## EDA Avançado: séries temporais por bairro e correlações

In [ ]:
# Rain 24h por bairro (média diária) — agregando para visualização mais limpa
try:
    tmp = df.set_index('date').groupby('neighborhood')['rain_24h_sum'].resample('1D').mean().reset_index()
except Exception:
    tmp = df.groupby(['neighborhood', pd.Grouper(key='date', freq='1D')])['rain_24h_sum'].mean().reset_index()

plt.figure(figsize=(12,6))
sns.lineplot(data=tmp, x='date', y='rain_24h_sum', hue='neighborhood')
plt.title('Rain 24h (média diária) por bairro')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
plt.show()

# Heatmap de correlação entre features numéricas
corr_cols = ['rain_mm','rain_24h_sum','rain_48h_sum','tide_cm','tide_12h_max','vuln_index','risk_index']
plt.figure(figsize=(8,6))
sns.heatmap(df[corr_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação entre features')
plt.show()

## Modelagem: RandomForest (com CV) e XGBoost opcional
- Preparar matriz X/y, codificar bairro por label encoding simples, treinar com StratifiedKFold e GridSearchCV leve.

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

features = ['rain_24h_sum','rain_48h_sum','tide_12h_max','rain_lag_1','hour','dayofweek','vuln_index']
# incluir neighborhood codificado
le = LabelEncoder()
df['nb_enc'] = le.fit_transform(df['neighborhood'])
features.append('nb_enc')

X = df[features].fillna(0).values
y = df['risk_label'].values

# dividir por tempo para evitar data leakage: usar último 20% como teste por data (opção rápida)
split_date = df['date'].quantile(0.8)
train_idx = df['date'] <= split_date
test_idx = df['date'] > split_date
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

print('Tamanhos:', X_train.shape, X_test.shape)

# Escalar features numéricas (opcional, RF não exige mas XGB pode se beneficiar)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# GridSearch leve para RandomForest (rápido) — ajustar conforme necessidade
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
param_grid = {'n_estimators':[50,100], 'max_depth':[5,10,None]}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
gs = GridSearchCV(rf, param_grid, cv=cv, scoring='roc_auc', n_jobs=1)
gs.fit(X_train, y_train)
best_rf = gs.best_estimator_
print('Best RF params:', gs.best_params_)

# Avaliar no conjunto de teste
y_proba = best_rf.predict_proba(X_test)[:,1]
y_pred = best_rf.predict(X_test)
print('AUC:', roc_auc_score(y_test, y_proba))
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, zero_division=0))
print('Recall:', recall_score(y_test, y_pred, zero_division=0))
print('Confusion matrix:
', confusion_matrix(y_test, y_pred))

# Salvar scaler + modelo rápido
models_dir = Path('..') / 'models'
models_dir.mkdir(parents=True, exist_ok=True)
joblib.dump(scaler, models_dir / 'scaler_advanced.joblib')
joblib.dump(best_rf, models_dir / 'rf_advanced.joblib')
print('Modelos salvos em', models_dir)

# Treinar XGBoost leve se disponível
if optional.get('xgboost'):
    xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    xgb_params = {'n_estimators':[50], 'max_depth':[3,6]}
    gs_xgb = GridSearchCV(xgb_clf, xgb_params, cv=cv, scoring='roc_auc', n_jobs=1)
    gs_xgb.fit(X_train, y_train)
    joblib.dump(gs_xgb.best_estimator_, models_dir / 'xgb_advanced.joblib')
    print('XGBoost salvo (melhor):', gs_xgb.best_params_)
else:
    print('XGBoost não foi treinado (lib ausente)')

## Explicabilidade com SHAP (se disponível)
- Executa SHAP summary plot para o RandomForest. Se não houver SHAP, mostra importâncias de features como fallback.

In [ ]:
if optional.get('shap'):
    explainer = shap.TreeExplainer(best_rf)
    # usar uma amostra para performance
    sample = X_test[np.random.choice(len(X_test), min(200, len(X_test)), replace=False)]
    shap_values = explainer.shap_values(sample)
    print('Plotando summary plot (pode demorar)')
    shap.summary_plot(shap_values, sample, feature_names=features)
else:
    print('SHAP não disponível — mostrando importâncias de features do RandomForest')
    importances = best_rf.feature_importances_
    feat_imp = pd.Series(importances, index=features).sort_values(ascending=False)
    print(feat_imp)
    feat_imp.plot(kind='barh')
    plt.title('Feature importances — RandomForest')
    plt.show()

## Visualização espacial (folium) — fallback para centroids se lat/lon ausentes

In [ ]:
# Tentar usar colunas lat/lon. Se não existirem, criar centroides fictícios por bairro (apenas para visualização)
if 'lat' in df.columns and 'lon' in df.columns and optional.get('folium'):
    m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
    for _, r in df.groupby('neighborhood').mean().reset_index().iterrows():
        folium.CircleMarker([r['lat'], r['lon']], radius=5, popup=r['neighborhood']).add_to(m)
    display(m)
else:
    # centroides fictícios (exemplo) e export para HTML (se folium disponível)
    centroids = {
        'Boa Viagem':(-8.1240,-34.9016),'Espinheiro':(-8.0435,-34.9011),'Casa Amarela':(-8.0142,-34.9231),
        'Ibura':(-8.1350,-34.9150),'Várzea':(-8.0625,-34.9032),'Santo Amaro':(-8.0609,-34.9056),
        'Centro':(-8.0640,-34.8711),'Aflitos':(-8.0440,-34.8890),'Pina':(-8.1235,-34.8811),'Cordeiro':(-8.0490,-34.8820)
    }
    nb_df = df.groupby('neighborhood')[['risk_index']].mean().reset_index()
    nb_df['lat'] = nb_df['neighborhood'].map(lambda x: centroids.get(x,(np.nan,np.nan))[0])
    nb_df['lon'] = nb_df['neighborhood'].map(lambda x: centroids.get(x,(np.nan,np.nan))[1])
    if optional.get('folium'):
        m = folium.Map(location=[nb_df['lat'].mean(), nb_df['lon'].mean()], zoom_start=12)
        for _, r in nb_df.iterrows():
            folium.CircleMarker([r['lat'], r['lon']], radius=5 + r['risk_index']*10, popup=f"{r['neighborhood']}: {r['risk_index']:.2f}", color='red' if r['risk_index']>0.5 else 'green').add_to(m)
        out_html = Path('..') / 'img' / 'risk_map.html'
        out_html.parent.mkdir(parents=True, exist_ok=True)
        m.save(str(out_html))
        print('Mapa salvo em', out_html)
    else:
        print('folium não disponível. Exibindo tabela de risco por bairro:')
        display(nb_df.sort_values('risk_index', ascending=False))

## Próximos passos e como executar
1) Instalar dependências opcionais para análises completas: xgboost, shap, folium.

2) Executar células sequencialmente.
3) Ajustar threshold de risco/target se houver labels reais e rodar validação temporal mais rígida.
4) Exportar modelos e criar endpoints para dashboard.


In [ ]:
print('Resumo: notebook criado com células para carregamento flexível, EDA, engenharia de features, modelos e explicabilidade.')
print('Modelos (se treinados) foram salvos em ../models por padrão.')